In [1]:
import string
import pickle
import re

import inflection
import nltk
from nltk.stem.porter import PorterStemmer

from assets import stop_words, java_keywords
from parsers import Parser
import glob
import os
import pandas as pd
from rank_bm25 import BM25Okapi, BM25Plus

ModuleNotFoundError: No module named 'inflection'

In [3]:
import numpy as np
b = np.load("/data/hdj/tracking_buggy_files/joblib_memmap_aspectj/code_urls.npy")
a=np.load("/data/hdj/tracking_buggy_files/joblib_memmap_aspectj/nl_urls.npy")

ModuleNotFoundError: No module named 'numpy'

In [4]:
import numpy as np

ModuleNotFoundError: No module named 'numpy'

In [3]:
src='/data/hdj/SourceFile/data/clean/sourceFile_aspectj/org.aspectj'
src_addresses = glob.glob(src + '/**/*.java', recursive=True)
len(src_addresses)

2357

In [ ]:
data=pd.read_csv('/data/hdj/cross_project_trans/report/aspectj_report_clean.csv')
data.head()

In [11]:
data[data['fixed_files']=="['org.aspectj/modules/org.aspectj.ajdt.core/src/org/aspectj/ajdt/internal/compiler/lookup/AjLookupEnvironment.java']"]

,bug_id,fixed_files,pos_tagged_descriptions,pos_tagged_summarys,descriptions,summarys
82,70773,['org.aspectj/modules/org.aspectj.ajdt.core/sr...,"{'stemmed': ['aspectj', 'eclips', 'build', 'id...","{'stemmed': ['aspectj', 'error', 'conncurrentm...","{'stemmed': ['aspectj', 'eclips', 'build', 'id...","{'stemmed': ['aspectj', 'error', 'conncurrentm..."
184,125810,['org.aspectj/modules/org.aspectj.ajdt.core/sr...,"{'stemmed': ['error', 'issu', 'implement', 'po...","{'stemmed': ['miss', 'error', 'inherit', 'poin...","{'stemmed': ['error', 'issu', 'sub', 'aspect',...","{'stemmed': ['miss', 'error', 'inherit', 'poin..."
192,128655,['org.aspectj/modules/org.aspectj.ajdt.core/sr...,"{'stemmed': ['version', 'aspectj', 'version', ...","{'stemmed': ['annot', 'compil', 'crash', 'decl...","{'stemmed': ['ajdt', 'version', 'aspectj', 've...","{'stemmed': ['annot', 'compil', 'crash', 'poss..."
235,148536,['org.aspectj/modules/org.aspectj.ajdt.core/sr...,"{'stemmed': ['aspect', 'bug', 'declar', 'type'...","{'stemmed': ['nullpointerexcept', 'declar', 'a...","{'stemmed': ['aspect', 'bug', 'declar', 'type'...","{'stemmed': ['nullpointerexcept', 'declar', 'i..."


In [10]:
not_duplicate=data['fixed_files'].values
duplicate=set(data['fixed_files'].values)
print('len :',len(not_duplicate),len(duplicate))
has_exist=set()
for string in not_duplicate:
#     print('not dup :',string)
    if string not in has_exist:
#         print(string)
        has_exist.add(string)
    else:
        print('已经出现了 :',string)

len : 286 264
已经出现了 : ['org.aspectj/modules/weaver/src/org/aspectj/weaver/bcel/LazyMethodGen.java']
已经出现了 : ['org.aspectj/modules/org.aspectj.ajdt.core/src/org/aspectj/ajdt/internal/core/builder/AjState.java']
已经出现了 : ['org.aspectj/modules/org.aspectj.ajdt.core/src/org/aspectj/tools/ajc/Main.java']
已经出现了 : ['org.aspectj/modules/runtime/src/org/aspectj/lang/SoftException.java']
已经出现了 : ['org.aspectj/modules/weaver/src/org/aspectj/weaver/bcel/BcelShadow.java']
已经出现了 : ['org.aspectj/modules/weaver/src/org/aspectj/weaver/AsmRelationshipProvider.java']
已经出现了 : ['org.aspectj/modules/org.aspectj.ajdt.core/src/org/aspectj/ajdt/internal/core/builder/EclipseAdapterUtils.java']
已经出现了 : ['org.aspectj/modules/asm/src/org/aspectj/asm/internal/ProgramElement.java']
已经出现了 : ['org.aspectj/modules/org.aspectj.ajdt.core/src/org/aspectj/ajdt/internal/compiler/ast/AspectDeclaration.java']
已经出现了 : ['org.aspectj/modules/weaver/src/org/aspectj/weaver/tools/WeavingAdaptor.java']
已经出现了 : ['org.aspectj/modules/w

In [11]:
len(set(data['bug_id'].values)),len(set(data['fixed_files'].values))

(286, 264)

In [14]:
class SrcPreprocessing_fulltext:
    def __init__(self,code_dir):
        self.src_files=self.get_repo_source_files(code_dir)

    def tokenize(self):
        """Tokenizing source codes into tokens"""
        for key,vals in self.src_files.items():
            tmp=[]
            for val in vals:
                tmp.append(nltk.wordpunct_tokenize(val))
            self.src_files[key]= tmp

    def _split_camelcase(self,tokens):

        # Copy tokens
        returning_tokens = tokens[:]

        for token in tokens:
            split_tokens = re.split(fr'[{string.punctuation}]+', token)

            # If token is split into some other tokens
            if len(split_tokens) > 1:
                returning_tokens.remove(token)
                # Camel case detection for new tokens
                for st in split_tokens:
                    camel_split = inflection.underscore(st).split('_')
                    if len(camel_split) > 1:
                        returning_tokens.append(st)
                        returning_tokens += camel_split
                    else:
                        returning_tokens.append(st)
            else:
                camel_split = inflection.underscore(token).split('_')
                if len(camel_split) > 1:
                    returning_tokens += camel_split

        return returning_tokens

    def split_camelcase(self):
        """Split CamelCase identifiers"""

        for key,val_list in self.src_files.items():
            tmp = []
            # print('vals',val_list)
            for vals in val_list:
                # print('vals',vals)
                tmp.append(self._split_camelcase(vals))
            self.src_files[key] = tmp
    def normalize(self):
        """Removing punctuation, numbers and also lowercase conversion"""

        # Building a translate table for punctuation and number removal
        punctnum_table = str.maketrans({c: None for c in string.punctuation + string.digits})

        for key,vals in self.src_files.items():
            tmp = []
            for val in vals:
                # print(val)
                content_punctnum_rem = [token.translate(punctnum_table)for token in val]
                tmp.append([token.lower() for token in content_punctnum_rem if token])
            self.src_files[key] = tmp
    def remove_stopwords(self):
        """Removing stop words from tokens"""

        for key, val_list in self.src_files.items():
            tmp=[]
            for val in val_list:
                after_removed=[token for token in val if token not in stop_words]
                if len(after_removed)>0:
                    tmp.append(after_removed)
            self.src_files[key] = tmp
    def remove_java_keywords(self):
        """Removing Java language keywords from tokens"""

        for key, val_list in self.src_files.items():
            tmp=[]
            for val in val_list:
                after_removed =[token for token in val  if token not in java_keywords]
                if len(after_removed) > 0:
                    tmp.append(after_removed)
            self.src_files[key] = tmp

    def stem(self):
        """Stemming tokens"""

        # Stemmer instance
        stemmer = PorterStemmer()

        for key, val_list in self.src_files.items():
            tmp = []
            for val in val_list:
                tmp.append(dict(zip(['stemmed', 'unstemmed'],[[stemmer.stem(token) for token in val], val])))
            self.src_files[key] = tmp
    def preprocess(self):
        """Run the preprocessing"""

#         self.pos_tagging()
        self.tokenize()
#         self.split_camelcase()
#         self.normalize()
        self.remove_stopwords()
#         self.remove_java_keywords()
        self.stem()
    def get_repo_source_files(self,code_dir):
        files = dict()
        start_dir = os.path.normpath(code_dir)  # 规范path字符串形式
        for dir_, dir_names, file_names in os.walk(start_dir):  # os.walk()方法用于通过在目录树中游走输出在目录中的文件名，向上或者向下
            for filename in [f for f in file_names if f.endswith(".java")]:
                src_name = os.path.join(dir_, filename)  # 所有java文件名
                try:
                    with open(src_name, encoding='utf-8', mode='r') as src_file:
                        lines = src_file.readlines()
                except Exception as e:
                    print(e)
                    continue
                file_key = src_name.split(start_dir)[1]
                file_key = file_key[len(os.sep):]  # os.sep 不用考虑是linux 的‘/’ 或者是windows的 ‘\’

                # 过滤掉 copyright
                new_lines = []
                beforeTag = True
                for line in lines:
                    line = line.lstrip().replace('\n', '')
                    if 'package' in line or 'import' in line or 'class' in line:
                        beforeTag = False
                    if beforeTag == False:
                        new_lines.append(line)

                # 分离code和description
                source = []
                # if file_key == 'org.aspectj\modules\\ajde\\testdata\examples\coverage\ModelCoverage.java':
                #     print('newline: ',new_lines)
                for line in new_lines:
                    # if file_key == 'org.aspectj\modules\\ajde\\testdata\examples\coverage\ModelCoverage.java':
                    #     print(line)
                    if line == "": continue
                    if line.startswith("@"): continue
                    if line.find('/*') >= 0: continue
                    if line.startswith('*'): continue
                    if line.find('*/') >= 0: continue
                    # if line.find('import')>=0:continue
                    if line.startswith('//'):
                        continue
                    else:
                        if line.find('//') >= 0:
                            index = line.index('//')
                            line = line[:index]
                        # source += (line + '\n')
                        source.append(line + '\n')

                # print(source,'\n******************************\n')
                # print(file_key)
                # if file_key=='org.aspectj\modules\\ajde\\testdata\examples\coverage\ModelCoverage.java':
                #     print('here ',file_key,source)
                files[file_key] = source
                # break
        # self.src_files=files

        # for path,code in files.items():
        #     f = open('hdj.java','w',encoding='utf-8')
        #     # print(code,type(code))
        #     f.write(''.join(code))
            # break
        # out = pd.DataFrame()
        # out['path'] = files.keys()
        # out['source'] = files.values()
        # out.to_csv('code.csv', index=False)  #
        # return out
        return  files

In [15]:
def mkdir(path):
    index = path.rindex('/')
    dir=path[:index]
    if not os.path.exists(dir):
        os.makedirs(dir)
def pathNorm(a,reg):
    index = a.index(reg)
    path = a[index:]
    # path = '.'.join(path.split('/'))
    return path
def test2():
    srcs = {'/data/hdj/SourceFile/data/sourceFile_aspectj/': 'org.aspectj',
           }
            # '/home/hdj/bug_localization/data/ZXing/ZXing-1.6': 'com/google/zxing'}
    clean_path=['/data/hdj/SourceFile/data/clean/sourceFile_aspectj']
    i = 0
    for src,reg in srcs.items():
        src_addresses = glob.glob(src + '/**/*.java', recursive=True)
        code_prep = SrcPreprocessing_fulltext(src)
        code_prep.preprocess()
        print('总共java文件数 ，处理后java文件数',len(src_addresses),len(code_prep.src_files))
        j=0
        for path,val_list in code_prep.src_files.items():
            path=pathNorm(path,reg)
            path=os.path.join(clean_path[i],path)
            # path=os.path.join(,path)
            mkdir(path)
            if len(val_list)==0:
                print(path,' is null')
                j+=1
                continue
            with open(path, 'w', encoding='utf-8') as f_out:
                for val in val_list:
                    f_out.write(' '.join(val['stemmed']) + '\n')
        print(reg,'共有',j,'个文件被过滤掉')
        i+=1
            #分为stemmed 和unstemmed文件写入，暂时一切从简
            # print(path)
            # index = path.rindex('.')
            # with open(path[:index] + '_Stemmed.java', 'w', encoding='utf-8') as f_out:
            #     for val in val_list:
            #         f_out.write(' '.join(val['stemmed']) + '\n')
            # with open(path[:index] + '_Unstemmed.java', 'w', encoding='utf-8') as f_out:
            #     for val in val_list:
            #         f_out.write(' '.join(val['unstemmed']) + '\n')
            # print(key,val)
            # break


if __name__ == '__main__':
    # main()
    test2()

'utf-8' codec can't decode byte 0xe9 in position 426: invalid continuation byte
总共java文件数 ，处理后java文件数 2394 2393
/data/hdj/SourceFile/data/clean/sourceFile_aspectj/org.aspectj/tests/decp/9052d5d AspectX.java  is null
/data/hdj/SourceFile/data/clean/sourceFile_aspectj/org.aspectj/tests/bugs170/pr376351/be063b8 RAj.java  is null
/data/hdj/SourceFile/data/clean/sourceFile_aspectj/org.aspectj/tests/bugs172/pr398588/96ebaae AspectB.java  is null
/data/hdj/SourceFile/data/clean/sourceFile_aspectj/org.aspectj/tests/multiIncremental/PR192877/inc1/src/9b68a31 Test.java  is null
/data/hdj/SourceFile/data/clean/sourceFile_aspectj/org.aspectj/tests/multiIncremental/PR192877/base/src/9b68a31 Test.java  is null
/data/hdj/SourceFile/data/clean/sourceFile_aspectj/org.aspectj/tests/multiIncremental/PR192877/base/src/9b68a31 Foo.java  is null
/data/hdj/SourceFile/data/clean/sourceFile_aspectj/org.aspectj/tests/multiIncremental/pr117209/base/src/551b9ca P.java  is null
/data/hdj/SourceFile/data/clean/sour

# 处理report信息

In [7]:
ids=[]
bug_id=[]
summary=[]
description=[]
report_time=[]
report_timestamp=[]
status=[]
commit=[]
commit_timestamp=[]
files=[]
with open('/data/hdj/SourceFile/data/sourceFile_aspectj/bugreport.txt','r',encoding='utf-8') as f_in:
    for i,line in enumerate(f_in):
        if i==0:
            continue
        print(line)
        contents=line.split('\t')
        #id	bug_id	summary	description	report_time	report_timestamp	status	commit	commit_timestamp	files
        print(contents)
        ids.append(contents[0])
        bug_id.append(contents[1])
        summary.append(contents[2])
        description.append(contents[3])
        files.append(list(files))
        break

1	423257	Bug 423257 LTW - java.lang.VerifyError: Bad return type with generics and local variables		2013-12-04 19:43:22	1386200000	resolved fixed	dd88d21	1386350000	org.aspectj.ajdt.core/src/org/aspectj/ajdt/internal/core/builder/AjState.java tests/bugs175/pr423257/AspectX.java tests/bugs175/pr423257/Test.java tests/src/org/aspectj/systemtest/AllTests17.java tests/src/org/aspectj/systemtest/ajc175/Ajc175Tests.java tests/src/org/aspectj/systemtest/ajc175/AllTestsAspectJ175.java tests/src/org/aspectj/systemtest/incremental/tools/MultiProjectIncrementalTests.java weaver/src/org/aspectj/weaver/bcel/asm/StackMapAdder.java	52:tests/src/org/aspectj/systemtest/incremental/tools/MultiProjectIncrementalTests.java 129:org.aspectj.ajdt.core/src/org/aspectj/ajdt/internal/core/builder/AjState.java 179:weaver/src/org/aspectj/weaver/bcel/asm/StackMapAdder.java 258:tests/src/org/aspectj/systemtest/AllTests17.java

['1', '423257', 'Bug 423257 LTW - java.lang.VerifyError: Bad return type with generics an

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

class GHM_Loss(nn.Module):
    def __init__(self, bins, alpha):
        super(GHM_Loss, self).__init__()
        self._bins = bins
        self._alpha = alpha
        self._last_bin_count = None

    def _g2bin(self, g):
        return torch.floor(g * (self._bins - 0.0001)).long()

    def _custom_loss(self, x, target, weight):
        raise NotImplementedError

    def _custom_loss_grad(self, x, target):
        raise NotImplementedError

    def forward(self, x, target):
        g = torch.abs(self._custom_loss_grad(x, target)).detach()

        bin_idx = self._g2bin(g)

        bin_count = torch.zeros((self._bins))
        for i in range(self._bins):
            bin_count[i] = (bin_idx == i).sum().item()

        N = (x.size(0) * x.size(1))

        if self._last_bin_count is None:
            self._last_bin_count = bin_count
        else:
            bin_count = self._alpha * self._last_bin_count + (1 - self._alpha) * bin_count
            self._last_bin_count = bin_count

        nonempty_bins = (bin_count > 0).sum().item()

        gd = bin_count * nonempty_bins
        gd = torch.clamp(gd, min=0.0001)
        beta = N / gd

        return self._custom_loss(x, target, beta[bin_idx])


class GHMC_Loss(GHM_Loss):
    def __init__(self, bins, alpha):
        super(GHMC_Loss, self).__init__(bins, alpha)

    def _custom_loss(self, x, target, weight):
        return F.binary_cross_entropy_with_logits(x, target, weight=weight)

    def _custom_loss_grad(self, x, target):
        return torch.sigmoid(x).detach() - target


In [2]:
ghm=GHMC_Loss(bins=30,alpha=0.75)

In [3]:

import numpy as np

batch_size = 8
class_num = 2
label = np.random.randint(0,class_num,size=(batch_size,1))
label = torch.LongTensor(label)
print(label)
y_one_hot = torch.zeros(batch_size,class_num).scatter_(1,label,1)
print(y_one_hot)

tensor([[1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1]])
tensor([[0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.]])


In [4]:
device='cuda'
pred = torch.randn((3,2), requires_grad=True)
target=torch.tensor([[0.0,1.0],[1.0,0.0],[1.0,0.0]])

# loss = F.binary_cross_entropy_with_logits(input, target)
# loss.backward()
pred=pred.to(device, dtype=torch.float).cuda(non_blocking=True)
target=target.to(device, dtype=torch.float).cuda(non_blocking=True)
print(pred.shape,target.shape)
print(pred)
print(target)

torch.Size([3, 2]) torch.Size([3, 2])
tensor([[ 0.2826, -0.2776],
        [ 0.5978,  0.8595],
        [-1.0763,  0.5448]], device='cuda:0', grad_fn=<CopyBackwards>)
tensor([[0., 1.],
        [1., 0.],
        [1., 0.]], device='cuda:0')


In [5]:
ghm=ghm.to(device)

In [6]:
print(type(ghm))

<class '__main__.GHMC_Loss'>


In [9]:
# loss=ghm(pred,target)
# print(loss)

In [3]:
pred=torch.tensor([[0.2,0.8],[0.1,0.9],[0.4,0.6]], requires_grad=True)
# pred=torch.tensor([[0.2],[0.9],[0.6]])
# target=torch.tensor([[0.0,1.0],[1.0,0.0],[1.0,0.0]])
target=torch.tensor([[0.0,1.],[1.,0.],[1.,1.]])
print(pred.shape,target.shape)
# loss=ghm(pred,target)
# print(loss)

torch.Size([3, 2]) torch.Size([3, 2])


In [10]:
input = torch.randn(3, requires_grad=True)
target = torch.empty(3).random_(2)
loss = F.binary_cross_entropy_with_logits(input, target)
loss.backward()

In [12]:
print(input.shape,target.shape)
input,target

torch.Size([3]) torch.Size([3])


(tensor([-2.2548, -0.7794, -0.3030], requires_grad=True), tensor([0., 0., 1.]))

In [12]:
from torch import nn
import torch
from torch.nn import functional as F

class focal_loss(nn.Module):
    def __init__(self, alpha=[0.25,0.75], gamma=2, num_classes = 2, size_average=False):
        """
        focal_loss损失函数, -α(1-yi)**γ *ce_loss(xi,yi)
        步骤详细的实现了 focal_loss损失函数.
        :param alpha: 阿尔法α,类别权重.      
                    当α是列表时,为各类别权重；
                    当α为常数时,类别权重为[α, 1-α, 1-α, ....],
                    常用于目标检测算法中抑制背景类, 
                    retainnet中设置为0.25
        :param gamma: 伽马γ,难易样本调节参数. retainnet中设置为2
        :param num_classes: 类别数量
        :param size_average: 损失计算方式,默认取均值
        """
        super(focal_loss,self).__init__()
        self.size_average = size_average
        if isinstance(alpha,list):
            assert len(alpha)==num_classes   
            # α可以以list方式输入,
            # size:[num_classes] 用于对不同类别精细地赋予权重
            print("Focal_loss alpha = {}, 将对每一类权重进行精细化赋值".format(alpha))
            self.alpha = torch.Tensor(alpha)
        else:
            assert alpha<1  #如果α为一个常数,则降低第一类的影响,在目标检测中为第一类
            print("Focal_loss alpha = {} ,将对背景类进行衰减,请在目标检测任务中使用.".format(alpha))
            self.alpha = torch.zeros(num_classes)
            self.alpha[0] += alpha
            self.alpha[1:] += (1-alpha) # α 最终为 [ α, 1-α, 1-α, 1-α, 1-α, ...] size:[num_classes]

        self.gamma = gamma

    def forward(self, preds, labels):
        """
        focal_loss损失计算
        :param preds: 预测类别. size:[B,N,C] or [B,C]    分
                别对应与检测与分类任务, B 批次, N检测框数, C类别数
        :param labels:  实际类别. size:[B,N] or [B]
        :return:
        """
        # assert preds.dim()==2 and labels.dim()==1
        preds = preds.view(-1,preds.size(-1))
#         print('preds :',preds)
        self.alpha = self.alpha.to(preds.device)
        # 这里并没有直接使用log_softmax, 因为后面会用到softmax的结果(当然也可以使用log_softmax,然后进行exp操作)
        preds_softmax = F.softmax(preds, dim=1) 
#         print('preds_softmax :',preds_softmax)
        preds_logsoft = torch.log(preds_softmax)
#         print('preds_logsoft :',preds_logsoft)        
        # 这部分实现nll_loss ( crossempty = log_softmax + nll )
        preds_softmax = preds_softmax.gather(1,labels.view(-1,1))   
#         print('preds_softmax :',preds_softmax) 
        preds_logsoft = preds_logsoft.gather(1,labels.view(-1,1))
#         print('preds_logsoft :',preds_logsoft) 
        self.alpha = self.alpha.gather(0,labels.view(-1))
        loss = -torch.mul(torch.pow((1-preds_softmax), self.gamma), preds_logsoft)  
        # torch.pow((1-preds_softmax), self.gamma) 为focal loss中 (1-pt)**γ

        loss = torch.mul(self.alpha, loss.t())
        if self.size_average:
            loss = loss.mean()
        else:
            loss = loss.sum()
        return loss

In [4]:

import torch
bert=torch.tensor([[0.2,0.8],[0.1,0.9],[0.4,0.6]])
fea=torch.tensor([[1],[2],[3]])
out=torch.cat((bert,fea),axis=1)
out

tensor([[0.2000, 0.8000, 1.0000],
        [0.1000, 0.9000, 2.0000],
        [0.4000, 0.6000, 3.0000]])

In [2]:
pred=torch.tensor([[0.2,0.8],[0.1,0.9],[0.4,0.6]], requires_grad=True)
# pred=torch.tensor([[0.2],[0.9],[0.6]])
# target=torch.tensor([[0.0,1.0],[1.0,0.0],[1.0,0.0]])
target=torch.tensor([[0],[1],[1]])

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=[0.25,0.75], gamma=2, size_average=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = torch.Tensor([gamma])
        self.size_average = size_average
        if isinstance(alpha, (float, int)):
            if self.alpha > 1:
                raise ValueError('Not supported value, alpha should be small than 1.0')
            else:
                self.alpha = torch.Tensor([alpha, 1.0 - alpha])
        if isinstance(alpha, list): self.alpha = torch.Tensor(alpha)
        self.alpha /= torch.sum(self.alpha)

    def forward(self, input, target):
        if input.dim() > 2:
            input = input.view(input.size(0), input.size(1), -1)  # [N,C,H,W]->[N,C,H*W] ([N,C,D,H,W]->[N,C,D*H*W])
        # target
        # [N,1,D,H,W] ->[N*D*H*W,1]
        if self.alpha.device != input.device:
            self.alpha = torch.tensor(self.alpha, device=input.device)
        target = target.view(-1, 1)
        logpt = torch.log(input + 1e-10)
        logpt = logpt.gather(1, target)
        logpt = logpt.view(-1, 1)
        pt = torch.exp(logpt)
        alpha = self.alpha.gather(0, target.view(-1))

        gamma = self.gamma

        if not self.gamma.device == input.device:
            gamma = torch.tensor(self.gamma, device=input.device)

        loss = -1 * alpha * torch.pow((1 - pt), gamma) * logpt
        if self.size_average:
            loss = loss.mean()
        else:
            loss = loss.sum()
        return loss

In [4]:
floss=FocalLoss()
loss=floss(pred,target)
loss

tensor(0.2164, grad_fn=<MeanBackward0>)

In [13]:
floss=focal_loss()
loss=floss(pred,target)

Focal_loss alpha = [0.25, 0.75], 将对每一类权重进行精细化赋值


In [14]:
loss

tensor(0.2258, grad_fn=<SumBackward0>)